In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Discrete, Box
import numpy as np
import scipy.stats as stats
import random
import time
import ray
from rllib_bandit.bandit.bandit import  BanditLinUCB, BanditLinUCBConfig, BanditLinTSConfig, BanditLinTS
from CustomDictObsEnv import CustomDictObsEnv, feaures_model_coefficient

In [2]:
ray.init(ignore_reinit_error=True, num_cpus=2)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-03-15 16:48:02,814	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.9.3
Dashboard:,http://127.0.0.1:8265


In [2]:
from environments.simulator_env_1 import SimulatorEnv1, LogisticModelFeature
from utils import FEATURES
INTERCEPT = 10
N = 10000
simulator_env = SimulatorEnv1(
    discrete_action_space=True,
    intercept=INTERCEPT,
    features=FEATURES,
    n_reps=N,
    first_arm=0.1,
    last_arm=0.9,)
input_data = simulator_env.get_obs_dataset(100)
input_data.head()

,apw,base_price,customer_type,hotel_level,night,occupancy,promotion
0,6,186.449962,1,2,1,2,3
1,72,199.390025,1,5,8,5,1
2,26,159.450858,1,4,2,2,0
3,0,175.077632,0,0,7,5,3
4,11,172.820999,1,5,6,2,3


In [3]:
input_data_copy = input_data.copy()

In [30]:


class FlattenObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservationWrapper, self).__init__(env)
        
        # Calculate the flat observation space size
        obs_space = env.observation_space
        self.flat_dim = sum(np.prod(obs_space.spaces[key].shape) for key in obs_space.spaces)
        
        # Define the new observation space (assuming all features are float64)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.flat_dim,), dtype=np.float64)
# [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    def observation(self, observation):
        # Flatten the dictionary observation into a single vector
        flat_obs = np.concatenate([np.array(observation[key]).flatten() for key in observation])
        return flat_obs


In [31]:
from ray.tune.registry import register_env
def create_env(env_config):
    # Initialize your custom environment with any necessary configuration
    env = CustomDictObsEnv(config=env_config)
    # Wrap the environment
    wrapped_env = FlattenObservationWrapper(env)
    return wrapped_env
register_env("custom_flat_env", create_env)

In [32]:
config_bandit =  BanditLinUCBConfig()\
         .environment(env='custom_flat_env', 
                      env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
                        disable_env_checking=True,
         )\
        .rollouts(num_rollout_workers=1)\
        .framework("tf2")


algo_sim= config_bandit.build()

for i in range(5):
    results = algo_sim.train()
    # print(f'Iter:{i} ; results = {results}')
    print(f"Iter: {i}; total reward={results['episode_reward_mean']}")


2024-03-15 16:40:57,167	WARNING util.py:62 -- Install gputil for GPU system monitoring.


Iter: 0; total reward=4918.013559478864
Iter: 1; total reward=4914.67913521538
Iter: 2; total reward=4831.659905156153
Iter: 3; total reward=4681.6000199026885
Iter: 4; total reward=4596.008070735507


In [33]:
# step by step 
env = CustomDictObsEnv(config={
    "intercept": 10.0,
    "n_reps": 100,
    "fetures_model_coefficient": feaures_model_coefficient,
    "data": input_data_copy,
})
wrapped_env = FlattenObservationWrapper(env)
obs, _= wrapped_env.reset()
for i in range(1):  # For each episode
    done = False
    total_reward = 0
    steps = 0
    wrapped_env.reset()
    while not done:
        # Example: Select an action using your algorithm (this is pseudo-code)
        
        action = algo_sim.compute_single_action(obs)
        
        # Take the action in the environment
        obs, reward, done, truncated, info = wrapped_env.step(action)
        adjust_fee = wrapped_env.actions[action]
        total_reward += reward
        
        steps += 1
        cumulative_avg_reward = total_reward / (steps )
        #print(f"Step {steps}:Observation= {obs}, Action={action},Fee = {adjust_fee:.2f}, Reward={reward}")
        if steps % 10 == 0:
            print(f"Step {steps}:Observation= {obs}, Action={action},Fee = {adjust_fee:.2f}, Reward={reward}")
            print(f"Cumulative Avg Reward={cumulative_avg_reward}")
    print(f"Iter: {i}; Total Reward={total_reward}")

Step 10:Observation= [ 21.         184.37962576   0.           3.           8.
   8.           0.        ], Action=8,Fee = 0.46, Reward=75.40036323861851
Cumulative Avg Reward=26.48002637857826
Step 20:Observation= [ 57.         172.00528395   1.           3.           6.
   1.           0.        ], Action=9,Fee = 0.50, Reward=0.0
Cumulative Avg Reward=44.56156644923211
Step 30:Observation= [  8.         157.71062937   1.           4.           6.
   3.          10.        ], Action=8,Fee = 0.46, Reward=72.2015212273428
Cumulative Avg Reward=45.476442666465246
Step 40:Observation= [ 13.         186.02626149   1.           1.           3.
   2.           1.        ], Action=5,Fee = 0.32, Reward=54.40314369362316
Cumulative Avg Reward=47.46680651427036
Step 50:Observation= [ 32.         164.11664237   0.           0.           2.
   5.           1.        ], Action=9,Fee = 0.50, Reward=101.38840332075495
Cumulative Avg Reward=47.06959735092352
Step 60:Observation= [ 15.         187.7213

In [178]:
from collections import OrderedDict
class ObservationBanditWrapper(gym.ObservationWrapper):

    def __init__(self, env):
        super().__init__(env)
        # Define custom low and high bounds for each dimension
        embedding_dim = len(env.observation_space.spaces)
        self.observation_space = spaces.Dict({
            "item": spaces.Box(low=0, high= np.inf,shape = ( 5, embedding_dim ), dtype=np.float64)
        })
        
    

    def observation(self, observation):
        # Convert the original observation dictionary into the desired format
        new_observation = np.concatenate([np.array(observation[key]).flatten() for key in observation])
        # reshaped_observation = new_observation.reshape( 1, -1)
        reshaped_observation = np.vstack([new_observation] * 5) 
        final_observation = {'item': reshaped_observation }
        # print("Final observation shape from wrapper:", final_observation['item'].shape)
        return final_observation 

In [179]:
env = CustomDictObsEnv(config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,}   )
ObservationBanditWrapper(env ).observation(env.observation_space.sample())['item'].shape

(5, 7)

In [180]:
env.observation_space.sample()

OrderedDict([('apw', array([60.89905416])),
             ('base_price', array([906.7442724])),
             ('customer_type', array([1.3173731])),
             ('hotel_level', array([5.76599628])),
             ('night', array([39.88228962])),
             ('occupancy', array([1.57918011])),
             ('promotion', array([5.25433449]))])

In [181]:
from ray.tune.registry import register_env
def create_env(env_config):
    # Initialize your custom environment with any necessary configuration
    env = CustomDictObsEnv(config=env_config)
    # Wrap the environment
    wrapped_env = ObservationBanditWrapper(env)
    return wrapped_env
register_env("custom_wrapper_env", create_env)

In [182]:


from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.tune.registry import register_env
from ray import tune

# Define your custom callback class
class MyCustomCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker, base_env,
                        episode, env_index, **kwargs):
        # Retrieve the last observation, action, reward, etc.
        print(f"Worker: {worker}, Env Index: {env_index}")
        
    def on_episode_step(
        self,
        *,
        worker,
        base_env,
        policies,
        episode,
        env_index,
        **kwargs,
    ):
        assert episode.length > 0, (
            "ERROR: `on_episode_step()` callback should not be called right "
            "after env reset!"
        )
        info = episode.last_info_for(env_index)
        print(f'show info: {info}')
        if info is not None:  # Make sure info is not None
            action = info.get('action')
            reward = info.get('reward')
            # Since observation is not directly available, it's trickier to log it here.
            # You might need to adjust your environment's 'info' to include the observation
            # or find another way to track it.
            print(f"Action: {action}, Reward: {reward}")

# Make sure

In [183]:
from ray import air, tune
config = (
    BanditLinUCBConfig()
    .framework("tf2")
    .environment(
        env='custom_wrapper_env', # noqa
        env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
         disable_env_checking=True,
    )
    .reporting(metrics_num_episodes_for_smoothing=500)
    .debugging(seed=0)
)

stop_reward = 180

tuner = tune.Tuner(
    BanditLinUCB,
    param_space=config.to_dict(),
    run_config=air.RunConfig(
        stop={
            "sampler_results/episode_reward_mean": stop_reward,
            "timesteps_total": 1500000,
        },
        failure_config=air.FailureConfig(fail_fast="raise"),
    ),
)
results = tuner.fit()

/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/tune/execution/tune_controller.py:190: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
  warnings.warn(


RayActorError: The actor died because of an error raised in its creation task, [36mray::BanditLinUCB.__init__()[39m (pid=81941, ip=127.0.0.1, actor_id=5f498932a0d93edabc308b6a01000000, repr=BanditLinUCB)
  File "/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 516, in __init__
    super().__init__(
  File "/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 638, in setup
    self.workers = WorkerSet(
  File "/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 159, in __init__
    self._setup(
  File "/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.

In [169]:
results


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'update_latency': 0.0023169517517089844}}}, 'num_env_steps_sampled': 100, 'num_env_steps_trained': 100, 'num_agent_steps_sampled': 100, 'num_agent_steps_trained': 100}, 'sampler_results': {'episode_reward_max': 1431.500881738094, 'episode_reward_min': 1431.500881738094, 'episode_reward_mean': 1431.500881738094, 'episode_len_mean': 99.0, 'episode_media': {}, 'episodes_this_iter': 1, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1431.500881738094], 'episode_lengths': [99]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.3209255709506498, 'mean_inference_ms': 0.8934441179332167, 'mean_action_processing_ms': 0.05288643412070699, 'mean_env_wait_ms': 0.4764334990246461, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_

In [172]:
config_bandit =  BanditLinUCBConfig()\
         .environment(env='custom_wrapper_env', 
                      env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
                        disable_env_checking=True,
         )\
        .rollouts(num_rollout_workers=1)\
        .framework("tf2")\
.callbacks(MyCustomCallbacks)

algo_sim= config_bandit.build()
checkpoint_path = "/Users/xizhang/ray_results/BanditLinUCB_2024-03-19_18-25-47/BanditLinUCB_custom_wrapper_env_1b494_00000_0_2024-03-19_18-25-47/checkpoint_000000"
algo_sim.restore(checkpoint_path)

2024-03-19 18:34:40,774	WARNING util.py:62 -- Install gputil for GPU system monitoring.
2024-03-19 18:34:40,845	INFO trainable.py:585 -- Restored on 127.0.0.1 from checkpoint: Checkpoint(filesystem=local, path=/Users/xizhang/ray_results/BanditLinUCB_2024-03-19_18-25-47/BanditLinUCB_custom_wrapper_env_1b494_00000_0_2024-03-19_18-25-47/checkpoint_000000)


In [153]:
config_bandit =  BanditLinUCBConfig()\
         .environment(env='custom_wrapper_env', 
                      env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
                        disable_env_checking=True,
         )\
        .rollouts(num_rollout_workers=1)\
        .framework("tf2")\
.callbacks(MyCustomCallbacks)

algo_sim= config_bandit.build()

for i in range(10):
    results = algo_sim.train()
    # print(f'Iter:{i} ; results = {results}')
    #print(results)
    print(f"Iter: {i}; total reward={results['episode_reward_mean']}")


/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/sit

UnsupportedSpaceException: Action space (Box(0.0, 10.0, (5,), float64)) of BanditTFPolicy_eager_traced is not supported for Bandit algorithms. Must be `Discrete`.

In [139]:
wrapper_env = create_env({
    "intercept": 10.0,
    "n_reps": 100,
    "fetures_model_coefficient": feaures_model_coefficient,
    "data": input_data_copy,
})

In [140]:
wrapper_env.observation(wrapper_env.observation_space.sample())["item"].shape

(5, 35)

In [177]:

# Get the initial observation (should be: [0.0] for the starting position).
obs, info = wrapper_env.reset()
terminated = truncated = False
total_reward = 0.0
# Play one episode.
while not terminated and not truncated:
    # Compute a single action, given the current observation
    # from the environment.
    action = algo_sim.compute_single_action(obs)
    # Apply the computed action in the environment.
    obs, reward, terminated, truncated, info = wrapper_env.step(action)
    # Sum up rewards for reporting purposes.
    total_reward += reward
    print(f"Observation= {obs} ,Action={action}, Reward={reward}, Total Reward={total_reward}")
# Report results.
print(f"Played 1 episode; total-reward={total_reward}")

ValueError: Expected flattened obs shape of [..., 35], got (1, 5, 7)

In [23]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from rllib_bandit.bandit.bandit_tf_model import OnlineLinearRegression
# Example feature dimensions and regularization parameters
feature_dim = 7
alpha = 1.0
lambda_ = 0.1

# Instantiate the OnlineLinearRegression model
model = OnlineLinearRegression(feature_dim=feature_dim, alpha=alpha, lambda_=lambda_)

# Simulate some incoming data
x = tf.convert_to_tensor(np.random.normal(size=(1, feature_dim)).astype(np.float32))  # Batch size 1
y = tf.convert_to_tensor(np.random.normal(size=(1,)).astype(np.float32))  # Single target value

# Perform a partial fit with the new data
model.partial_fit(x, y)

# Predict the output for a new input, also converted to a tensor
new_x = tf.convert_to_tensor(np.random.normal(size=(1, feature_dim)).astype(np.float32))
prediction = model(new_x, sample_theta=False)  # Using mean parameters for prediction
print(f"Prediction: {prediction.numpy()}")

Prediction: [0.6233386]


In [26]:
input_dict  = wrapper_env.observation(wrapper_env.observation_space.sample())
print(input_dict)

{'item': array([[1.29246571, 2.54275292, 0.68691875, 0.08373766, 0.41089234,
        0.36165869, 0.20613969]])}


In [85]:

from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_tf, try_import_tfp
from ray.rllib.utils.typing import TensorType

class ParametricLinearModel(TFModelV2):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TFModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )

        alpha = model_config.get("alpha", 1)
        lambda_ = model_config.get("lambda_", 0.1)

        # RLlib preprocessors will flatten the observation space and unflatten
        # it later. Accessing the original space here.
        
        if not hasattr(obs_space, 'original_space'):
            obs_space.original_space = obs_space
            original_space = obs_space.original_space
            print("Original space:", original_space)
        else:
            
            assert (
                isinstance(original_space, gym.spaces.Dict)
                and "item" in original_space.spaces
            ), "This model only supports gym.spaces.Dict observation spaces."
        self.feature_dim = original_space["item"].shape[-1]
       
        self.arm = OnlineLinearRegression(
            feature_dim=self.feature_dim, alpha=alpha, lambda_=lambda_
        )
       
        self._cur_value = None
        self._cur_ctx = None

    def _check_inputs(self, x):
        assert (
            len(x.shape) == 3
        ), f"ERROR: Inputs ({x}) must have 3 dimensions (B x num-items x features)."
        return x

    @override(ModelV2)
    def forward(self, input_dict, state, seq_lens):
        x = input_dict["obs"]["item"]
        print(x)
        x = self._check_inputs(x)
        scores = self.predict(x)
        return scores, state

    def predict(self, x, sample_theta=False, use_ucb=False):
        self._cur_ctx = x
        scores = self.arm(x, sample_theta)
        
        if use_ucb:
            scores += 0.3 * self.arm.get_ucbs(x)
        self._cur_value = scores
        
        return scores

    def partial_fit(self, x, y, arms):
        x = x["item"]
        for i, arm in enumerate(arms):
            xi = tf.expand_dims(x[i, arm], axis=0)
            yi = tf.expand_dims(y[i], axis=0)
            self.arm.partial_fit(xi, yi)

    @override(ModelV2)
    def value_function(self):
        assert self._cur_value is not None, "Must call `forward()` first."
        return self._cur_value

    def current_obs(self):
        assert self._cur_ctx is not None, "Must call `forward()` first."
        return self._cur_ctx


class ParametricLinearModelUCB(ParametricLinearModel):
    def forward(self, input_dict, state, seq_lens):
        x = input_dict["obs"]["item"]
        x = self._check_inputs(x)
        scores = super().predict(x, sample_theta=False, use_ucb=True)
        print('ParametricLinearModelUCB scores:', scores)
        return scores, state


In [86]:

obs_space= ObservationBanditWrapper(env).observation_space
action_space = wrapper_env.action_space
model_config = {"alpha": 1.0, "lambda_": 0.1}
P_model = ParametricLinearModelUCB(obs_space, action_space, 1, model_config,   name="LinearModel")

Original space: Dict('item': Box(0.0, inf, (1, 7), float64))


In [90]:
input_dict['item']  = ObservationBanditWrapper(env).observation(env.observation_space.sample())['item'].reshape(1, 1, 7)

In [184]:
from recsim.simulator import environment
from recsim.simulator import recsim_gym
from recsim.environments import interest_exploration

# Define a simple user model.
user_model = interest_exploration.IEUserModel(
    slate_size=2,
    seed=0
)

# Define a document sampler to generate a set of documents from which the recommender can choose.
document_sampler = interest_exploration.IETopicDocumentSampler(seed=0)

# Number of documents to present to the agent at each step.
num_candidates = 10

# Create the RecSim environment.
recsim_env = environment.SingleUserEnvironment(
    user_model,
    document_sampler,
    num_candidates,
    slate_size=2,  # Number of recommendations to make at each step.
    resample_documents=True
)

# Wrap the RecSim environment in a Gym environment.
gym_env = recsim_gym.RecSimGymEnv(recsim_env)

print("Observation Space:", gym_env.observation_space)
print("Action Space:", gym_env.action_space)


TypeError: response_model_ctor is a required callable